In [110]:
import database
import req
import pprint
import json
import os
import csv
pwd = os.getcwd()
pp = pprint.PrettyPrinter(indent=4)

dbsavefile = '/DBsave.json'
#DB = database.DB_init()

def saveDB(db):
    with open("".join([pwd,dbsavefile]), 'wb') as jsonfile: 
            json.dump(db,jsonfile)

def num(s):
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return s

def d_or_v(val):
    if isinstance(val,dict):
        return key_to_int(val)
    else:
        return val
   
def key_to_int(d):
    return {num(k):d_or_v(v) for k,v in d.iteritems()}   

def loadDB():
    with open("".join([pwd,dbsavefile]), 'rb') as jsonfile: 
        db = json.load(jsonfile)
    return key_to_int(db)

    

DB=loadDB()
pprint = pp.pprint
Teams = DB['Teams']
Games = DB['Games']



In [133]:

def print_team(d,score = 0.0):
    pprint(''.join([str(d['id']),': ',d['name'],'  strength = ',str(score)]))

def print_game(d):
    if d['status']==1:
        pprint("".join([str(d['id']),':', 'r', str(d['round']),'  ',
                        str(d['scores'][d['team_1']]),' -- ',
                        DB['Teams'][d['team_1']]['name'],
                        ' vs ', DB['Teams'][d['team_2']]['name'], ' -- ',
                        str(d['scores'][d['team_2']])]))
    else:
        pprint("".join([str(d['id']),':','r',str(d['round']),'  ',
                DB['Teams'][d['team_1']]['name'],
                ' vs ', DB['Teams'][d['team_2']]['name']]))
def list_teams(session_id):
    session = DB['Sessions'][session_id]
    for id in session['teams']:
        s = session['strengths'][id]
        print_team(DB['Teams'][id],score = s)
def list_games(session_id):
    session = DB['Sessions'][session_id]
    for id in session['games']:
        print_game(DB['Games'][id])

        
def games_played(team_id,session_id):
    def count(g,t):
        if DB['Games'][g]['status'] == 1 and t in [DB['Games'][g]['team_1'],DB['Games'][g]['team_2']]:
            return 1
        else:
            return 0
        
    return sum([count(g_id,team_id) for g_id in DB['Sessions'][session_id]['games']])
    
def export_results(session_id):
    session = DB['Sessions'][session_id]
    
    
    d = sorted([[id,DB['Teams'][id]['name'],
          session['strengths'][id],
         games_played(id,session_id)] for id in session['teams']],key = lambda x: x[2])

    with open("".join([pwd,'/results.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in d:
            writer.writerow(line)

            
def export_games(session_id):
    session = DB['Sessions'][session_id]
    played_games = [Games[id] for id in session['games'] if Games[id]['status'] == 1]
    unplayed_games = [Games[id] for id in session['games'] if not Games[id]['status'] == 1]

    pd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round'],g['scores'][g['team_1']],
            g['scores'][g['team_2']]] for g in played_games]    
    upd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round'],'none',
            'none'] for g in unplayed_games]    
    with open("".join([pwd,'/games.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        
        for line in upd:
           writer.writerow(line)
        
def export_suggested_games(session_id):
    session = DB['Sessions'][session_id]
    suggested_games = session['suggestedGames'].values()

    pd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round']] for g in suggested_games]    

    with open("".join([pwd,'/suggested_games.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        
def export_teams_list(session_id):
    session = DB['Sessions'][session_id]
    teams = [DB['Teams'][id] for id in session['teams']]

    pd = [[t['id'],t['name']] for t in teams]    

    with open("".join([pwd,'/team_list.csv']),'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        




In [166]:
sess_id = 3
export_teams_list(sess_id)
export_games(sess_id)

def compute_strengths():
    req.update_strengths(DB,sess_id)
    export_results(sess_id)

def stage_round(round = 0):
    req.get_suggested_games(DB,sess_id,round)
    export_suggested_games(sess_id)
    print('done')
    
def update_score(game_id=-1,score_1 = -1, score_2 = -1):
    req.update_score(DB,game_id,score_1,score_2)
    export_games(sess_id)
    print_game(Games[game_id])

def add_suggested_games():
    req.add_suggested_games(DB,sess_id)
    export_games(sess_id)
def add_suggested_game(game_id=0):
    req.add_game(DB,sess_id,DB['Sessions'][sess_id]['suggestedGames'][game_id])
    export_games(sess_id)

def add_game(team_1 = 0, team_2 = 0, round = 0):
    req.add_game(DB,sess_id,req.new_game(team_1,team_2,round))
    export_games(sess_id)

def remove_game(game_id=-1):
    req.remove_game(DB, sess_id, game_id)
    export_games(sess_id)

    
def check_teams(team_1=0,team_2=0):
    games = [Games[id] for id in DB['Sessions'][sess_id]['games']]
    for game in games:
        if set([game['team_1'],game['team_2']]) == set([team_1,team_2]):
            print('played')
            return true
    print('unplayed')
    return False

In [154]:
#Add a game to the session. Use the team ID to specify the teams
add_game(team_1 = 0, team_2 = 27, round = 7 )

In [168]:
# update the scores in a game. get the game id from the list of games
update_score(game_id=113,score_1 =4, score_2 =0)

u'113:r7  4 -- Mammoth vs Lost Cat -- 0'


In [72]:
# remove a game from the session
remove_game(game_id=-1)

In [169]:
# Compute the strengths of the teams. This may take some time.
compute_strengths()

[9, 13, 7, 10, 12, 3, 5, 25, 24, 17, 23, 14, 20, 26, 13, 9, 7, 12, 3, 5, 10, 12, 13, 9, 5, 7, 3, 10, 7, 13, 10, 5, 3, 12, 9, 12, 10, 5, 9, 13, 7, 3, 24, 14, 25, 23, 20, 26, 17, 24, 25, 20, 14, 23, 17, 26, 20, 25, 24, 23, 14, 17, 26, 14, 26, 25, 17, 24, 20, 23]
[0, 1, 2, 4, 6, 8, 11, 15, 16, 18, 19, 21, 22, 27]
running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 49.0 sec

In [170]:
#Suggest games for the next round. This may take quite some time.
stage_round(round  =8)

[9, 13, 7, 10, 12, 3, 5, 25, 24, 17, 23, 14, 20, 26, 13, 9, 7, 12, 3, 5, 10, 12, 13, 9, 5, 7, 3, 10, 7, 13, 10, 5, 3, 12, 9, 12, 10, 5, 9, 13, 7, 3, 24, 14, 25, 23, 20, 26, 17, 24, 25, 20, 14, 23, 17, 26, 20, 25, 24, 23, 14, 17, 26, 14, 26, 25, 17, 24, 20, 23]
[0, 1, 2, 4, 6, 8, 11, 15, 16, 18, 19, 21, 22, 27]
running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 49.2 sec[0, 1, 3, 5, 7, 10, 14, 15, 17, 18, 20, 21, 26, 27, 29, 31, 33, 36, 40, 41, 43, 44, 46, 47, 52, 54, 56, 58, 61, 65, 66, 68, 69, 71, 72, 77, 103, 105, 108, 112, 113, 115, 116, 118, 119, 124, 148, 151, 155, 156, 158, 159, 161, 162, 167, 190, 194, 195, 197, 198, 200, 201, 206, 245, 246, 248, 249, 251, 252, 257, 300, 302, 303, 305, 306, 311, 313, 314, 316, 317, 322, 333, 335, 336, 341, 343, 344, 349, 357, 362, 367]
[9, 13, 7, 10, 12, 3, 5, 25, 24, 17, 23, 14, 20, 26, 13, 9, 7, 12, 3, 5, 10, 12, 13, 9, 5, 7, 3, 10, 7, 13, 10, 5, 3, 12, 9, 12, 10, 5, 9, 13, 7, 3, 24, 14, 25, 23, 20, 26, 17, 24, 

In [359]:
#This will add the suggested games to the list of games
add_suggested_games()

In [161]:
#add one of the suggested games to the list of games
add_suggested_game(game_id=0)

In [172]:
check_teams(team_1=1,team_2=91)

unplayed


False

In [171]:
saveDB(DB)

In [134]:
export_results(sess_id)

In [224]:
export_games(sess_id)

In [198]:
export_games(1)

In [126]:
import bayes
reload(req)
reload(bayes)
import ranking
reload(ranking)

<module 'ranking' from 'ranking.py'>

unplayed


NameError: global name 'false' is not defined

In [42]:
DB['Sessions'][3]['games'] = DB['Sessions'][1]['games'] + DB ['Sessions'][4]['games']
DB['Sessions'][4]['games'] = DB['Sessions'][1]['games'] + DB ['Sessions'][4]['games']

DB['Sessions'][3]['teams'] = [2,19,1,16,11,18,6,15,8,21,0,22,4,27]
DB['Sessions'][4]['teams'] = [24,13,14,9,25,7,17,12,23,3,20,10,26,5]

In [141]:
import bayes
import ranking

teams = bayes.make_teams(DB['Sessions'][3]['teams'])
sess = bayes.init_session(teams)
sess.games = bayes.mcmc_games([DB['Games'][id] for id in DB['Sessions'][3]['games']],teams)


[9, 13, 7, 10, 12, 3, 5, 25, 24, 17, 23, 14, 20, 26, 13, 9, 7, 12, 3, 5, 10, 12, 13, 9, 5, 7, 3, 10, 7, 13, 10, 5, 3, 12, 9, 12, 10, 5, 9, 13, 7, 3, 24, 14, 25, 23, 20, 26, 17, 24, 25, 20, 14, 23, 17, 26, 20, 25, 24, 23, 14, 17, 26, 14, 26, 25, 17, 24, 20, 23]
[0, 1, 2, 4, 6, 8, 11, 15, 16, 18, 19, 21, 22, 27]


In [142]:
sess.fit_model()


running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 48.2 sec

In [143]:
kl = sess.kl_info_vec()


[0, 1, 3, 5, 7, 10, 14, 15, 17, 18, 20, 21, 26, 27, 29, 31, 33, 36, 40, 41, 43, 44, 46, 47, 52, 54, 56, 58, 61, 65, 66, 68, 69, 71, 72, 77, 103, 105, 108, 112, 113, 115, 116, 118, 119, 124, 148, 151, 155, 156, 158, 159, 161, 162, 167, 190, 194, 195, 197, 198, 200, 201, 206, 245, 246, 248, 249, 251, 252, 257, 300, 302, 303, 305, 306, 311, 313, 314, 316, 317, 322, 333, 335, 336, 341, 343, 344, 349, 357, 362, 367]


In [308]:
sess.stage_round(3,kl)

array([-1.62747166,  0.6548901 , -0.26899063, -1.81805333, -2.54009176,
       -1.26907531,  0.9887355 ,  1.17901925,  3.02039566,  0.43067404,
       -2.61601751, -1.2592841 ,  2.0404588 ,  3.08481095])

In [145]:
kl

[{'t1': 0, 't2': 1, 'weight': 0.043488434978840917},
 {'t1': 0, 't2': 2, 'weight': 0.052911956334933397},
 {'t1': 0, 't2': 4, 'weight': 1.423212044740469},
 {'t1': 0, 't2': 6, 'weight': 0.47233959100775591},
 {'t1': 0, 't2': 8, 'weight': 0.97590125420481799},
 {'t1': 0, 't2': 11, 'weight': 0.063336122891256785},
 {'t1': 0, 't2': 15, 'weight': 1.4539676426516679},
 {'t1': 0, 't2': 16, 'weight': 0.59508049599408563},
 {'t1': 0, 't2': 18, 'weight': 0.5797376045887348},
 {'t1': 0, 't2': 19, 'weight': 0.3013815259912529},
 {'t1': 0, 't2': 21, 'weight': 1.6675247396431692},
 {'t1': 0, 't2': 22, 'weight': 1.5773338307057236},
 {'t1': 0, 't2': 27, 'weight': 2.1925986893425509},
 {'t1': 1, 't2': 2, 'weight': 1.8153443317916067},
 {'t1': 1, 't2': 4, 'weight': 0.29056747308138098},
 {'t1': 1, 't2': 6, 'weight': 0.51421833902599179},
 {'t1': 1, 't2': 8, 'weight': 0.11852944873525839},
 {'t1': 1, 't2': 11, 'weight': 0.52874216984817757},
 {'t1': 1, 't2': 15, 'weight': 0.13905401374219561},
 {'t1': 

In [272]:
sess.games[1].teams[0].id

5

In [116]:
g= sess.games[0]
[t.id for t in g.teams]

[2, 9]

In [328]:
x=req.update_strengths(DB,2)

running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 30.1 sec

In [329]:
x['Sessions'][2]['strengths']

{0: 1.240902108482105,
 1: -1.7313163712697726,
 2: -2.8765658394250044,
 3: 1.1288194903466013,
 4: 1.4651319018689322,
 5: 2.4161783699511941,
 6: 0.14419265791853592,
 7: -0.053151918012899063,
 8: 0.96596172161713834,
 9: -1.1502022887057295,
 10: 1.2421464553926431,
 11: -1.0857910719168931,
 12: 0.40310069577556018,
 13: -2.1094059120224267,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0}

In [115]:
DB['Sessions'][4]['games'] = list(set(DB['Sessions'][4]['games']
                                 + DB['Sessions'][1]['games']
                                 + DB ['Sessions'][2]['games']))

In [2]:
pwd

'/home/ubuntu/bayesian/python'

In [16]:

DB['Sessions'][3]['teams'] = [2,19,1,16,11,18,6,15,8,21,0,22,4,27]
DB['Sessions'][4]['teams'] = [24,13,14,9,25,7,17,12,23,3,20,10,26,5]
DB['Sessions'][3]['strengths'] = {k:0.0 for k in DB['Sessions'][3]['teams']}
DB['Sessions'][4]['strengths'] = {k:0.0 for k in DB['Sessions'][4]['teams']}

In [144]:
saveDB(DB)

In [51]:
list_teams(3)

u'2: Golem  strength = 0.0'
u'19: Beware  strength = 0.0'
u'1: Wabbit  strength = 0.0'
u'16: The Jamesons  strength = 0.0'
u'11: Triple Fisted  strength = 0.0'
u'18: Mammoth  strength = 0.0'
u'6: Gentlemans  strength = 0.0'
u'15: The Royales with Cheese  strength = 0.0'
u'8: Rusty Spoons  strength = 0.0'
u'21: Lost Cat  strength = 0.0'
u'0: Ball Bangers  strength = 0.0'
u'22: Mighty Bisons  strength = 0.0'
u'4: JKL  strength = 0.0'
u"27: The Caracal's  strength = 0.0"


In [19]:
list_teams(4)

u'24: Three Boys  strength = 0.0'
u'13: Pete un Plomb  strength = 0.0'
u'14: Kitten Pile Crush  strength = 0.0'
u'9: Daft Monks  strength = 0.0'
u'25: Gobshites  strength = 0.0'
u'7: Sons of Odin  strength = 0.0'
u'17: Elf Death  strength = 0.0'
u'12: With Fire  strength = 0.0'
u'23: Sometimes  strength = 0.0'
u'3: The Edinburgh Badgers  strength = 0.0'
u'20: Two and a half Men  strength = 0.0'
u'10: The Budgie Smugglers  strength = 0.0'
u'26: Kumaras bravas  strength = 0.0'
u'5: The Inseminators  strength = 0.0'


In [55]:
export_games(4)

In [104]:
session['games']

{u'games': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  42,
  43,
  45,
  46,
  102,
  103,
  104,
  105,
  106,
  107,
  108],
 u'id': 3,
 u'name': u'Wildcard: Day 2 AM',
 u'strengths': {0: 0.0,
  1: 0.0,
  2: 0.0,
  4: 0.0,
  6: 0.0,
  8: 0.0,
  11: 0.0,
  15: 0.0,
  16: 0.0,
  18: 0.0,
  19: 0.0,
  21: 0.0,
  22: 0.0,
  27: 0.0},
 u'suggestedGames': [],
 u'teams': [2, 19, 1, 16, 11, 18, 6, 15, 8, 21, 0, 22, 4, 27],
 u'type': u'swiss'}